# Generate stores & customers for the demo

## Setup the environment

### Install packages

In [2]:
!pip install faker
!pip install pymorton
!pip install v3io==0.1.1

### Define environment variables

In [212]:
NUMBER_OF_STORES=500
STORES_TABLE='stores'
ACCURACY=20
BASE_ACCURACY=14

## Function code

### imports

In [274]:
import logging
import os
import pickle
import random
import itertools

# Demo
import pandas as pd
import pymorton as pm
from faker import Faker
from faker.providers import BaseProvider

# DB
import v3io as v3c
import v3io_frames as v3f

### Location provider

In [215]:
class LocationProvider(BaseProvider):
    '''
    Creates locations within base_location

    Uses QuadTree for Geohashing
        @{http://tech.taskrabbit.com/blog/2015/06/09/elasticsearch-geohash-vs-geotree/}
        @{http://mapzen.github.io/leaflet-spatial-prefix-tree/}
        @{http://blog.notdot.net/2009/11/Damn-Cool-Algorithms-Spatial-indexing-with-Quadtrees-and-Hilbert-Curves}
    '''    
    def location(self, location_base: str, base_acc=10, acc=20):
        coordinates = location_base[:base_acc]
        for i in range(acc-len(coordinates)):
            coordinates += str(random.randint(0, 3))
        return coordinates

Add the location provider as a faker provider

In [216]:
faker = Faker()
faker.add_provider(LocationProvider)

### Define scenario environment (locations)

In [303]:
london_city = (51.514926, -0.089580)
london_city_south = (51.501593, -0.094942)
london_west = (51.512309, -0.128966)
london_south_west = (51.495022, -0.162268)

london_coordinates = []
london_coordinates.append(london_city)
london_coordinates.append(london_city_south)
london_coordinates.append(london_west)
london_coordinates.append(london_south_west)

london_coordinates_qt = list(map(lambda cooridnate: pm.interleave_latlng(*cooridnate), london_coordinates))
london_coordinates_qt

['03311311313011311011000321002320',
 '03311311311233323013031101320003',
 '03311311313010023000032330133111',
 '03311311311222300331010333220231']

### Define Store class

In [219]:
class Store():
    def __init__(self, store_id: int, name: str, location: str):
        self.store_id = store_id if store_id is not None else random.randint(1, int(os.genev('NUMBER_OF_STORES', 500)))
        self.location = location
        self.store_name = name
    
    def json(self):
        json_store = {
            'id': self.store_id,
            'name': self.store_name,
            'location': self.location
        }
        return json_store

In [304]:
def create_stores(faker, number_of_stores: int=500):
    remaining = number_of_stores
    stores = []
    while remaining > 0:
        # Create stores
        stores_batch = [Store(faker.msisdn(), faker.company(), faker.location(random.choice(london_coordinates_qt), BASE_ACCURACY, ACCURACY)).json() 
              for i in range(remaining)]
        
        # Count for duplicate locations
        locations = list(map(lambda store: store['location'], stores_batch))
        locations = list(dict.fromkeys(locations))
        remaining -= len(locations)
        
        # Update stores list
        stores.append(stores_batch)

    stores = list(itertools.chain.from_iterable(stores))
    stores = pd.DataFrame.from_records(stores)
    stores = stores.set_index(['location'])
    stores = stores[~stores.index.duplicated(keep='first')]
    return stores

In [305]:
stores = create_stores(faker, NUMBER_OF_STORES)
stores.head(5)

,id,name
location,,
03311311311233012230,2793819899349,Jones LLC
03311311311233133123,9677713513610,"Thompson, Jones and White"
03311311313011033100,4407028136334,Jackson-Smith
03311311313011120331,5466349386394,Smith Inc
03311311313011220201,1008467073137,"Burgess, Greene and Rivera"


### Upload to DB 

In [224]:
client = v3f.Client('http://'+os.getenv('V3IO_FRAMESD'))

In [301]:
# client.delete('kv', STORES_TABLE)
client.write('kv', STORES_TABLE, stores)

{'num_frames': 1, 'num_rows': 500}

In [302]:
client.read('kv', STORES_TABLE)

,id,name
__name,,
03311311313010131133,7183910721301,"Walker, Robertson and Moore"
03311311313011233230,8017707548500,Perry-Lee
03311311311222333012,1151444597730,"Horton, Phillips and Blair"
03311311313011320300,0092378625034,Carlson LLC
03311311311222203223,4676773693752,"Johnson, Hopkins and Ballard"
03311311313010231211,6967390061337,Walker-Lopez
03311311313011132322,9378889406089,"Mann, Myers and Bridges"
03311311311233301011,6260797135382,"Nicholson, Smith and Hamilton"
03311311311222011302,6086073120471,Boone Group
